In [1]:
# --- AoC 2025 - Day 12: Christmas Tree Farm ---

# 0. Configuration and Imports
# -----------------------------------------------------------------------------
import os
import sys
import re
from collections import defaultdict, Counter, deque
import math
import itertools
import heapq

# Define input filename - typically both parts use the same 'input.txt'
INPUT_FILENAME = "input.txt"

# Set up paths for convenience
NOTEBOOK_DIR = os.getcwd() 


# 1. Load Input Data
# -----------------------------------------------------------------------------
def load_input_data(filename):
    """
    Loads input data from a specified file.
    Assumes the input file is in the same directory as the notebook.
    """
    filepath = os.path.join(NOTEBOOK_DIR, filename)
    try:
        with open(filepath, 'r') as f:
            # Read all lines and strip whitespace from each
            return [line.rstrip() for line in f.readlines()]
    except FileNotFoundError:
        print(f"Error: Input file '{filename}' not found at '{filepath}'")
        return [] # Return an empty list to prevent further errors

# Load the raw data once for both parts
raw_data = load_input_data(INPUT_FILENAME)

if raw_data:
    print(f"Loaded {len(raw_data)} lines from '{INPUT_FILENAME}'.")
    print(f"First 5 lines: {raw_data[:5]}\n")
else:
    print("No data loaded (or file not found). Operations will rely on test data.\n")


# =============================================================================
# >>> START PART 1 (Solve this first!) <<<
# =============================================================================

# 2. Part 1 Data Preprocessing / Parsing
# -----------------------------------------------------------------------------
def normalize_shape(coords):
    """
    Shifts coordinates so top-left is at (0,0).
    Returns a sorted tuple of coordinates.
    """
    if not coords:
        return tuple()
    min_r = min(r for r, c in coords)
    min_c = min(c for r, c in coords)
    return tuple(sorted((r - min_r, c - min_c) for r, c in coords))

def generate_variations(base_shape_coords):
    """
    Generates all unique rotation/flip variations of a shape.
    base_shape_coords: list of (r, c)
    """
    variations = set()
    
    # 4 rotations
    current = base_shape_coords
    for _ in range(4):
        # Rotate 90 degrees clockwise: (r, c) -> (c, -r)
        current = [(c, -r) for r, c in current]
        variations.add(normalize_shape(current))
        
    # Flip (e.g., horizontal flip: (r, c) -> (r, -c))
    flipped = [(r, -c) for r, c in base_shape_coords]
    current = flipped
    for _ in range(4):
        current = [(c, -r) for r, c in current]
        variations.add(normalize_shape(current))
        
    return list(variations)

def parse_data_part1(data_lines):
    """
    Parses the raw input data for Part 1.
    Output:
    - shapes: Dict {id: [list of variation tuples]}
    - regions: List of dicts {'w': int, 'h': int, 'presents': [shape_id, ...]}
    """
    print("Parsing data for Part 1...")
    if not data_lines: 
        return {}, []

    shapes = {}
    regions = []
    
    iterator = iter(data_lines)
    
    # 1. Parse Shapes
    # Shapes are separated by blank lines, or end of section.
    # We don't have a clear delimiter between sections in the file structure 
    # other than the change in format.
    # Shapes start with "N:"
    # Regions start with "WxH:"
    
    current_shape_id = None
    current_shape_grid = []
    
    # Buffer lines to handle lookahead/mode switching
    buffer = []
    
    mode = "SHAPES" 
    
    for line in data_lines:
        line = line.strip()
        if not line:
            # End of a shape definition probably
            if current_shape_id is not None and current_shape_grid:
                # Process shape
                coords = []
                for r, row_str in enumerate(current_shape_grid):
                    for c, char in enumerate(row_str):
                        if char == '#':
                            coords.append((r, c))
                
                # Generate variations
                vars = generate_variations(coords)
                shapes[current_shape_id] = vars
                
                current_shape_id = None
                current_shape_grid = []
            continue
            
        # Check if line indicates a region (e.g., "12x5: ...")
        if 'x' in line and ':' in line and not line.startswith('#') and not line.startswith('.'):
            # It's a region line! switch mode logic, but actually we can just parse it directly.
            # If we were building a shape, finish it.
            if current_shape_id is not None and current_shape_grid:
                coords = []
                for r, row_str in enumerate(current_shape_grid):
                    for c, char in enumerate(row_str):
                        if char == '#':
                            coords.append((r, c))
                vars = generate_variations(coords)
                shapes[current_shape_id] = vars
                current_shape_id = None
                current_shape_grid = []
            
            # Parse Region
            # Format: "4x4: 0 0 0 0 2 0"
            dim_part, counts_part = line.split(':')
            w_str, h_str = dim_part.split('x')
            w, h = int(w_str), int(h_str)
            
            counts = list(map(int, counts_part.strip().split()))
            
            # Expand counts to list of present IDs
            present_list = []
            for shape_id, count in enumerate(counts):
                present_list.extend([shape_id] * count)
                
            regions.append({
                'w': w,
                'h': h,
                'presents': present_list
            })
            
        elif ':' in line and (line[0].isdigit()):
            # Start of a new shape: "0:"
            current_shape_id = int(line.split(':')[0])
            current_shape_grid = []
        else:
            # Shape content or junk
            if current_shape_id is not None:
                current_shape_grid.append(line)

    # Catch last shape if file ended without newline
    if current_shape_id is not None and current_shape_grid:
        coords = []
        for r, row_str in enumerate(current_shape_grid):
            for c, char in enumerate(row_str):
                if char == '#':
                    coords.append((r, c))
        vars = generate_variations(coords)
        shapes[current_shape_id] = vars

    return shapes, regions

parsed_shapes, parsed_regions = parse_data_part1(raw_data)
# print(f"Parsed {len(parsed_shapes)} shapes and {len(parsed_regions)} regions.")


# 3. Part 1 Solution Algorithm
# -----------------------------------------------------------------------------

def can_fit(region_w, region_h, presents, shape_defs, occupied, memo=None):
    """
    Backtracking solver.
    presents: list of shape_ids remaining to be placed.
    shape_defs: dict of shape_id -> variations.
    occupied: set of (r, c) tuples.
    """
    if not presents:
        return True
        
    # Heuristic: Sort presents by size (largest first) to fail fast?
    # This is done in the caller usually, but recursion handles the list order.
    
    current_shape_id = presents[0]
    remaining_presents = presents[1:]
    
    # Optimization: Check if identical to previous shape to break symmetry
    # We can pass a 'start_index' if needed, but for now simple backtracking.
    
    variations = shape_defs[current_shape_id]
    
    # Try all positions
    # To optimize: The shape must fit within (0..h-1, 0..w-1)
    
    # Symmetry breaking for identical shapes:
    # If current_shape_id == previous_shape_id, enforce placement index ordering.
    # This requires passing state. For AoC inputs, simpler might be fast enough.
    
    for var in variations:
        var_h = max(r for r, c in var) + 1
        var_w = max(c for r, c in var) + 1
        
        # Valid top-left positions
        for r in range(region_h - var_h + 1):
            for c in range(region_w - var_w + 1):
                
                # Check collision
                fits = True
                new_cells = []
                for dr, dc in var:
                    nr, nc = r + dr, c + dc
                    if (nr, nc) in occupied:
                        fits = False
                        break
                    new_cells.append((nr, nc))
                
                if fits:
                    # Place
                    for cell in new_cells:
                        occupied.add(cell)
                    
                    if can_fit(region_w, region_h, remaining_presents, shape_defs, occupied):
                        return True
                        
                    # Backtrack
                    for cell in new_cells:
                        occupied.remove(cell)
                        
    return False

# Optimized Solver with symmetry breaking
def solve_recursive(region_w, region_h, presents, shape_defs, occupied, present_idx, shape_areas):
    if present_idx == len(presents):
        return True

    # Pruning: Remaining area
    occupied_count = len(occupied)
    total_area = region_w * region_h
    remaining_area_needed = sum(shape_areas[pid] for pid in presents[present_idx:])
    
    if (total_area - occupied_count) < remaining_area_needed:
        return False

    current_pid = presents[present_idx]
    variations = shape_defs[current_pid]
    
    # Symmetry breaking: If this present is same ID as previous, start search after previous pos
    # To do this efficiently, we'd need to track the 'index' of the placement (r*w + c).
    # Since we aren't passing that, we'll stick to standard order.
    # Given limited computation time, let's just loop.
    
    for var in variations:
        # Optimization: precompute var dimensions
        var_h = max(r for r, c in var) + 1
        var_w = max(c for r, c in var) + 1
        
        for r in range(region_h - var_h + 1):
            for c in range(region_w - var_w + 1):
                # Optimization: Check one pixel first
                if (r + var[0][0], c + var[0][1]) in occupied:
                    continue

                # Check full collision
                fits = True
                new_cells = []
                for dr, dc in var:
                    nr, nc = r + dr, c + dc
                    if (nr, nc) in occupied:
                        fits = False
                        break
                    new_cells.append((nr, nc))
                
                if fits:
                    for cell in new_cells: occupied.add(cell)
                    
                    if solve_recursive(region_w, region_h, presents, shape_defs, occupied, present_idx + 1, shape_areas):
                        return True
                    
                    for cell in new_cells: occupied.remove(cell)

    return False

def solve_part1(parsed_data):
    """
    Solves the first part of the puzzle.
    """
    print("Solving Part 1...")
    shapes, regions = parsed_data
    
    # Precompute shape areas for pruning
    shape_areas = {}
    for sid, vars in shapes.items():
        if vars:
            shape_areas[sid] = len(vars[0])
        else:
            shape_areas[sid] = 0

    success_count = 0
    
    for i, reg in enumerate(regions):
        # Sort presents largest to smallest to fail fast
        presents = sorted(reg['presents'], key=lambda pid: shape_areas[pid], reverse=True)
        
        # Check simple area constraint first
        total_needed = sum(shape_areas[pid] for pid in presents)
        if total_needed > reg['w'] * reg['h']:
            continue
            
        occupied = set()
        if solve_recursive(reg['w'], reg['h'], presents, shapes, occupied, 0, shape_areas):
            success_count += 1
            # print(f"Region {i} fits.")
        else:
            # print(f"Region {i} fails.")
            pass
            
    return success_count

part1_answer = solve_part1((parsed_shapes, parsed_regions))
print(f"Part 1 Answer: {part1_answer}\n")


# 4. Part 1 Testing
# -----------------------------------------------------------------------------
EXAMPLE_INPUT_PART1_STR = """
0:
###
##.
##.

1:
###
##.
.##

2:
.##
###
##.

3:
##.
###
##.

4:
###
#..
###

5:
###
.#.
###

4x4: 0 0 0 0 2 0
12x5: 1 0 1 0 2 2
12x5: 1 0 1 0 3 2
"""
EXAMPLE_EXPECTED_PART1 = 2

def test_part1():
    print("Running Part 1 example test...")
    if not EXAMPLE_INPUT_PART1_STR.strip():
        print("Skipping test: No example data provided yet.")
        return

    example_raw_data = EXAMPLE_INPUT_PART1_STR.strip().split('\n')
    example_parsed_data = parse_data_part1(example_raw_data)
    result = solve_part1(example_parsed_data)
    
    assert result == EXAMPLE_EXPECTED_PART1, \
        f"Part 1 Example Failed! Expected {EXAMPLE_EXPECTED_PART1}, Got {result}"
    print(f"Test Result: {result} (Expected: {EXAMPLE_EXPECTED_PART1})")
    print("Part 1 example test finished.\n")

# Run the test
test_part1()


# =============================================================================
# >>> START PART 2 (Only after solving Part 1) <<<
# =============================================================================

# 5. Part 2 Data Preprocessing / Parsing
# -----------------------------------------------------------------------------
def parse_data_part2(data_lines):
    """
    Parses the raw input data for Part 2.
    """
    print("Parsing data for Part 2...")
    if not data_lines:
        return []

    # Usually reuses Part 1 parsing
    return parse_data_part1(data_lines)

parsed_input_part2 = parse_data_part2(raw_data)


# 6. Part 2 Solution Algorithm
# -----------------------------------------------------------------------------
def solve_part2(data):
    """
    Solves the second part of the puzzle.
    """
    print("Solving Part 2...")
    
    # --- Part 2 Algorithm Here ---

    return "Part 2 Solution Not Implemented Yet"

part2_answer = solve_part2(parsed_input_part2)
print(f"Part 2 Answer: {part2_answer}\n")


# 7. Part 2 Testing
# -----------------------------------------------------------------------------
EXAMPLE_INPUT_PART2_STR = EXAMPLE_INPUT_PART1_STR
EXAMPLE_EXPECTED_PART2 = 0

def test_part2():
    print("Running Part 2 example test...")
    if not EXAMPLE_INPUT_PART2_STR.strip():
        print("Skipping test: No example data provided yet.")
        return

    example_raw_data = EXAMPLE_INPUT_PART2_STR.strip().split('\n')
    example_parsed_data = parse_data_part2(example_raw_data)
    result = solve_part2(example_parsed_data)
    
    # assert result == EXAMPLE_EXPECTED_PART2, \
    #     f"Part 2 Example Failed! Expected {EXAMPLE_EXPECTED_PART2}, Got {result}"
    print(f"Test Result: {result} (Expected: {EXAMPLE_EXPECTED_PART2})")
    print("Part 2 example test finished.\n")

# Run the Part 2 test
# test_part2()

print("\n✨🎄 Join the event https://adventofcode.com/2025, let's code before Christmas")

Loaded 1030 lines from 'input.txt'.
First 5 lines: ['0:', '#.#', '###', '#.#', '']

Parsing data for Part 1...
Solving Part 1...
Part 1 Answer: 414

Running Part 1 example test...
Parsing data for Part 1...
Solving Part 1...
Test Result: 2 (Expected: 2)
Part 1 example test finished.

Parsing data for Part 2...
Parsing data for Part 1...
Solving Part 2...
Part 2 Answer: Part 2 Solution Not Implemented Yet


✨🎄 Join the event https://adventofcode.com/2025, let's code before Christmas
